<!--<badge>--><a href="https://colab.research.google.com/github/gradsflow/gradsflow/blob/main/examples/nbs/01-ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

First, install gradsflow

`pip install git+https://github.com/gradsflow/gradsflow@main`

In [ ]:
import os
import sys
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms as T

from gradsflow import AutoImageClassifier
from gradsflow.data.common import random_split_dataset

Let's use `CalTech101` dataset provided by `torchvision`

In [ ]:
# Replace dataloaders with your custom dataset and you are all set to train your model
image_size = (64, 64)
batch_size = 4

to_rgb = lambda x: x.convert("RGB")

augs = T.Compose([to_rgb, T.AutoAugment(), T.Resize(image_size), T.ToTensor()])
data = torchvision.datasets.Caltech101("~/", download=True, transform=augs)

Files already downloaded and verified


In [ ]:
train_data, val_data = random_split_dataset(data, 0.01)
train_dl = DataLoader(train_data, batch_size=batch_size)
val_dl = DataLoader(val_data, batch_size=batch_size)
num_classes = len(data.categories)

If you want to run Gradsflow on a remote server then first setup [ray cluster](https://docs.ray.io/en/master/cluster/index.html) and initialize ray with the remote address.

In [ ]:
# ray.init(address="REMOTE_IP_ADDR")
# ray.init(local_mode=True)

To train an image classifier create an object of `AutoImageClassifier` and provide number of trials and timeout.

In [ ]:
model = AutoImageClassifier(
    train_dataloader=train_dl,
    val_dataloader=val_dl,
    num_classes=num_classes,
    max_epochs=2,
    optimization_metric="train_loss",
    n_trials=4,
)
print("AutoImageClassifier initialised!")

AutoImageClassifier initialised!


In [ ]:
analysis = model.hp_tune()
print("completed!")

2022-01-09 14:50:33,186	WARNING function_runner.py:563 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,backbone,lr,optimizer
optimization_objective_657f0_00000,PENDING,,ssl_resnet18,0.00333062,adam


(pid=2443) /Users/aniket/miniconda3/envs/am/lib/python3.9/site-packages/gradsflow/utility/common.py:124: UserWarning: to_item didn't convert any value.
(pid=2443)   warnings.warn("to_item didn't convert any value.")


Trial name,status,loc,backbone,lr,optimizer
optimization_objective_657f0_00000,RUNNING,127.0.0.1:2443,ssl_resnet18,0.00333062,adam


Trial name,status,loc,backbone,lr,optimizer
optimization_objective_657f0_00000,RUNNING,127.0.0.1:2443,ssl_resnet18,0.00333062,adam


...

Trial name,status,loc,backbone,lr,optimizer
optimization_objective_657f0_00000,RUNNING,127.0.0.1:2443,ssl_resnet18,0.00333062,adam


Result for optimization_objective_657f0_00000:
  date: 2022-01-09_14-52-48
  done: false
  experiment_id: f14d18631a8942e392e8b69780f61d85
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 2443
  should_checkpoint: true
  time_since_restore: 132.34348511695862
  time_this_iter_s: 132.34348511695862
  time_total_s: 132.34348511695862
  timestamp: 1641720168
  timesteps_since_restore: 0
  train_accuracy: 0.005975634484635094
  train_loss: 5.911113500595093
  training_iteration: 1
  trial_id: 657f0_00000
  val_accuracy: 0.0
  val_loss: 137.33717279238851
  
Result for optimization_objective_657f0_00000:
  date: 2022-01-09_14-52-48
  done: true
  experiment_id: f14d18631a8942e392e8b69780f61d85
  experiment_tag: 0_backbone=ssl_resnet18,lr=0.0033306,optimizer=adam
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 2
  node_ip: 127.0.0.1
  pid: 2443
  should_checkpoint: true
  time_since_restore: 132.47166109085083
  time_this

Trial name,status,loc,backbone,lr,optimizer,iter,total time (s),val_loss,train_loss,train_accuracy
optimization_objective_657f0_00000,TERMINATED,127.0.0.1:2443,ssl_resnet18,0.00333062,adam,2,132.472,137.337,5.91111,0.00597563


2022-01-09 14:52:48,404	INFO tune.py:630 -- Total run time: 135.22 seconds (135.06 seconds for the tuning loop).


completed!


In [ ]:
# ray.shutdown()